<a href="https://colab.research.google.com/github/Jaesu26/vime/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VIME Example

`-` An example to train VIME-Self and VIME-Semi using google colab gpu

## Install VIME

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/vime

/content/drive/MyDrive/Colab Notebooks/vime


In [3]:
!pip install git+https://github.com/Jaesu26/vime.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Jaesu26/vime.git to /tmp/pip-req-build-252xpxpp
  Running command git clone --filter=blob:none --quiet https://github.com/Jaesu26/vime.git /tmp/pip-req-build-252xpxpp
  Resolved https://github.com/Jaesu26/vime.git to commit ef83f6be4ce3edfeb45bf37e6b03321a6b6b93df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 52.9 MB/s eta

## Prepare MNIST

In [4]:
!pip install easydict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import random
import warnings

import easydict
import numpy as np
import torch
import torch.nn as nn

warnings.filterwarnings("ignore") 

- Hyperparameters

In [6]:
args_mlp = easydict.EasyDict({
    "weights_dirpath": "./mlp_weights",
    "num_classes": 10,
    "max_epochs": 50,
    "batch_size": 64,
    "train_size": 0.9,
    "lr": 1e-3, 
    "log_interval": 5,
    "seed": 26,
})
args_self = easydict.EasyDict({
    "weights_dirpath": "./vimeself_weights",
    "max_epochs": 50,
    "batch_size": 512,
    "train_size": 0.9,
    "lr": 1e-2, 
    "p_masking": 0.3,
    "alpha": 2.0,
    "log_interval": 5,
    "seed": 26,
})
args_semi = easydict.EasyDict({
    "weights_dirpath": "./vimesemi_weights",
    "num_classes": 10,
    "supervised_criterion": nn.CrossEntropyLoss(),
    "max_epochs": 50,
    "labeled_batch_size": 64,
    "unlabeled_batch_size": 512,
    "train_size": 0.9,
    "lr": 1e-3, 
    "p_masking": 0.3,
    "K": 3,
    "beta": 1.0,
    "log_interval": 5,
    "seed": 26,
})

In [7]:
def create_folder(path: str) -> None:
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError as error:
        print(error)

In [8]:
create_folder(args_mlp.weights_dirpath)
create_folder(args_self.weights_dirpath)
create_folder(args_semi.weights_dirpath)

- Load data

In [9]:
import sklearn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [10]:
mnist = fetch_openml("mnist_784")

In [11]:
data = mnist.data.values
target = mnist.target.astype(int).values

In [12]:
data /= 255.0

In [13]:
data.shape

(70000, 784)

- Split data

In [14]:
num_labeled_data_used = 1000
unlabeled_data_rate = 0.9
seed = 26

In [15]:
X, X_test, y, y_test = train_test_split(data, target, test_size=1/7, random_state=seed, stratify=target)

In [16]:
X_labeled, X_unlabeled, y, _ = train_test_split(X, y, test_size=unlabeled_data_rate, random_state=seed, stratify=y)

In [17]:
X_labeled = X_labeled[:num_labeled_data_used]
y = y[:num_labeled_data_used]

## Supervised Model

In [18]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
from vime.datamodules import LabeledDataModule
from vime.lightningmodules import MLPClassifier

- Create datamodule and model

In [19]:
dim = X_labeled.shape[1]

In [20]:
labeled_datamodule = LabeledDataModule(
    X_labeled, y, X_test,
    train_size=args_mlp.train_size,
    batch_size=args_mlp.batch_size,
    seed=args_mlp.seed,
)

In [21]:
mlp_classifier = MLPClassifier(
    input_dim=dim,
    hidden_dims=[512, 256, 128],
    num_classes=args_mlp.num_classes,
    lr=args_mlp.lr,
    log_interval=args_mlp.log_interval,
    seed=args_mlp.seed,
)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


- Train supervised model

In [22]:
checkpoint = ModelCheckpoint(
    dirpath=args_mlp.weights_dirpath,
    filename="mlp",
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
)
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode="min",
)
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=False,
    callbacks=[checkpoint, early_stop],
    max_epochs=args_mlp.max_epochs,
    num_sanity_val_steps=0,
    enable_progress_bar=False, 
    enable_model_summary=False,
    deterministic=True,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(mlp_classifier, labeled_datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val Loss: 1.5906 | Val Macro Acc: 0.6977
Epoch 1 | Train Loss: 1.0895  Val Loss: 0.5416 | Val Macro Acc: 0.8029
Epoch 5 | Train Loss: 0.1458  Val Loss: 0.5540 | Val Macro Acc: 0.8152
Epoch 10 | Train Loss: 0.1133  Val Loss: 0.5382 | Val Macro Acc: 0.8274
Epoch 15 | Train Loss: 0.0826  Val Loss: 0.4827 | Val Macro Acc: 0.8662
Epoch 20 | Train Loss: 0.0374  Val Loss: 0.6857 | Val Macro Acc: 0.7724
Epoch 25 | Train Loss: 0.1387  Val Loss: 0.6290 | Val Macro Acc: 0.7959
Epoch 30 | Train Loss: 0.0981  

- Test supervised model

In [24]:
pred = trainer.predict(mlp_classifier, labeled_datamodule, ckpt_path="best")
pred = np.concatenate(pred).argmax(1)

INFO: Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp-v8.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp-v8.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp-v8.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp-v8.ckpt


In [25]:
accuracy_score(y_test, pred)

0.8883

## VIME

In [26]:
from vime import VIMESelf, VIMESelfDataModule, VIMESemi, VIMESemiDataModule

### VIME Self

- Create datamodule and model

In [27]:
dim = X_unlabeled.shape[1]

In [28]:
self_datamodule = VIMESelfDataModule(
    X_unlabeled,
    train_size=args_self.train_size,
    batch_size=args_self.batch_size,
    seed=args_self.seed,
)

In [29]:
vime_self = VIMESelf(
    input_dim=dim,
    hidden_dims=[512, 256, 128],
    lr=args_self.lr,
    p_masking=args_self.p_masking,
    alpha=args_self.alpha,
    log_interval=args_self.log_interval,
    seed=args_self.seed,
)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


- Train vime self

In [30]:
checkpoint = ModelCheckpoint(
    dirpath=args_self.weights_dirpath,
    filename="vime_self",
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
)
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode="min",
)
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=False,
    callbacks=[checkpoint, early_stop],
    max_epochs=args_self.max_epochs,
    num_sanity_val_steps=0,
    enable_progress_bar=False, 
    enable_model_summary=False,
    deterministic=True,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [31]:
trainer.fit(vime_self, self_datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val Loss: 0.1562 | Val Loss_m: 0.0957 | Val Loss_r: 0.0302
Epoch 1 | Train Loss: 0.3436 | Train Loss_m: 0.2508 | Train Loss_r: 0.0464  Val Loss: 0.1155 | Val Loss_m: 0.0839 | Val Loss_r: 0.0158
Epoch 5 | Train Loss: 0.2534 | Train Loss_m: 0.2179 | Train Loss_r: 0.0178  Val Loss: 0.1105 | Val Loss_m: 0.0842 | Val Loss_r: 0.0132
Epoch 10 | Train Loss: 0.2463 | Train Loss_m: 0.2156 | Train Loss_r: 0.0154  Val Loss: 0.1018 | Val Loss_m: 0.0783 | Val Loss_r: 0.0118
Epoch 15 | Train Loss: 0.2420 | Train Loss_m: 0.2124 | Train Loss_r: 0.0148  Val Loss: 0.1007 | Val Loss_m: 0.0771 | Val Loss_r: 0.0118
Epoch 20 | Train Loss: 0.2381 | Train Loss_m: 0.2091 | Train Loss_r: 0.0145  Val Loss: 0.0977 | Val Loss_m: 0.0744 | Val Loss_r: 0.0117
Epoch 25 | Train Loss: 0.2353 | Train Loss_m: 0.2064 | Train Loss_r: 0.0145  Val Loss: 0.0979 | Val Loss_m: 0.0738 | Val Loss_r: 0.0120
Epoch 30 | Train Loss: 0.2334 | Train Loss_m: 0.2042 | Train Loss_r: 0.0146  Val Loss: 0.0936 | Val Loss_m: 0.0702 | Val Loss_r

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Val Loss: 0.0915 | Val Loss_m: 0.0675 | Val Loss_r: 0.0120
Epoch 50 | Train Loss: 0.2290 | Train Loss_m: 0.1997 | Train Loss_r: 0.0147  

In [32]:
best_model_path = checkpoint.best_model_path

In [33]:
vime_self_best = VIMESelf.load_from_checkpoint(best_model_path)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


In [34]:
pretrained_encoder = vime_self_best.encoder

- Train supervised model from pretrained encoder

In [35]:
with torch.no_grad():
    Z = pretrained_encoder(torch.tensor(X_labeled, dtype=torch.float32).cuda())
    Z_test = pretrained_encoder(torch.tensor(X_test, dtype=torch.float32).cuda())

In [36]:
Z = Z.cpu().numpy()
Z_test = Z_test.cpu().numpy()

In [37]:
labeled_datamodule_from_unsupervised = LabeledDataModule(Z, y, Z_test, train_size=args_mlp.train_size, batch_size=args_mlp.batch_size, seed=args_mlp.seed)

In [38]:
mlp_classifier = MLPClassifier(
    input_dim=128,
    hidden_dims=[64, 32],
    num_classes=args_mlp.num_classes,
    lr=args_mlp.lr,
    seed=args_mlp.seed,
)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


In [39]:
checkpoint = ModelCheckpoint(
    dirpath=args_mlp.weights_dirpath,
    filename="mlp_from_unsupervised",
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
)
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode="min",
)
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=False,
    callbacks=[checkpoint, early_stop],
    max_epochs=args_mlp.max_epochs,
    num_sanity_val_steps=0,
    enable_progress_bar=False, 
    enable_model_summary=False,
    deterministic=True,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [40]:
trainer.fit(mlp_classifier, labeled_datamodule_from_unsupervised)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val Loss: 2.2118 | Val Macro Acc: 0.1578
Epoch 1 | Train Loss: 2.0487  Val Loss: 0.6566 | Val Macro Acc: 0.8467
Epoch 10 | Train Loss: 0.5176  Val Loss: 0.4887 | Val Macro Acc: 0.8617
Epoch 20 | Train Loss: 0.2015  Val Loss: 0.4214 | Val Macro Acc: 0.8679
Epoch 30 | Train Loss: 0.1038  Val Loss: 0.4393 | Val Macro Acc: 0.8728
Epoch 40 | Train Loss: 0.0750  

- Test vime self

In [41]:
pred = trainer.predict(mlp_classifier, labeled_datamodule_from_unsupervised, ckpt_path="best")
pred = np.concatenate(pred).argmax(1)

INFO: Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp_from_unsupervised-v1.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp_from_unsupervised-v1.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp_from_unsupervised-v1.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/mlp_weights/mlp_from_unsupervised-v1.ckpt


In [42]:
accuracy_score(y_test, pred)

0.8983

### VIME Semi

- Create datamodule and model

In [43]:
dim = X_labeled.shape[1]

In [44]:
semi_datamodule = VIMESemiDataModule(
    X_unlabeled,
    X_labeled,
    y,
    X_test,
    train_size=args_semi.train_size,
    labeled_batch_size=args_semi.labeled_batch_size,
    unlabeled_batch_size=args_semi.unlabeled_batch_size,
    seed=args_semi.seed,
)

In [45]:
vime_semi = VIMESemi(
    pretrained_encoder=pretrained_encoder,
    hidden_dims=[512, 256, 128],
    num_classes=args_semi.num_classes,
    supervised_criterion=args_semi.supervised_criterion,
    lr=args_semi.lr,
    p_masking=args_semi.p_masking,
    K=args_semi.K,
    beta=args_semi.beta,
    log_interval=args_semi.log_interval,
    seed=args_semi.seed,
)

INFO: Global seed set to 26
INFO:lightning.fabric.utilities.seed:Global seed set to 26


- Train vime semi

In [46]:
checkpoint = ModelCheckpoint(
    dirpath=args_semi.weights_dirpath,
    filename="vime_semi",
    monitor="val_loss",
    mode="min",
    save_weights_only=True,
)
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=20,
    mode="min",
)

In [47]:
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=False,
    callbacks=[checkpoint, early_stop],
    max_epochs=args_semi.max_epochs,
    num_sanity_val_steps=0,
    enable_progress_bar=False, 
    enable_model_summary=False,
    deterministic=True,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [48]:
trainer.fit(vime_semi, semi_datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val Loss_s: 1.2711
Epoch 1 | Train Loss: 1.2933 | Train Loss_s: 1.2183 | Train Loss_u: 0.0749  Val Loss_s: 0.4205
Epoch 5 | Train Loss: 0.3440 | Train Loss_s: 0.2292 | Train Loss_u: 0.1149  Val Loss_s: 0.4209
Epoch 10 | Train Loss: 0.2554 | Train Loss_s: 0.1380 | Train Loss_u: 0.1174  Val Loss_s: 0.4465
Epoch 15 | Train Loss: 0.2519 | Train Loss_s: 0.1401 | Train Loss_u: 0.1117  Val Loss_s: 0.4063
Epoch 20 | Train Loss: 0.2080 | Train Loss_s: 0.0985 | Train Loss_u: 0.1095  Val Loss_s: 0.4454
Epoch 25 | Train Loss: 0.2183 | Train Loss_s: 0.1045 | Train Loss_u: 0.1138  Val Loss_s: 0.4739
Epoch 30 | Train Loss: 0.2012 | Train Loss_s: 0.0911 | Train Loss_u: 0.1101  Val Loss_s: 0.4263
Epoch 35 | Train Loss: 0.1735 | Train Loss_s: 0.0715 | Train Loss_u: 0.1019  

- Test vime semi

In [49]:
pred = trainer.predict(vime_semi, semi_datamodule, ckpt_path="best")
pred = np.concatenate(pred).argmax(1)

INFO: Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/MyDrive/Colab Notebooks/vime/vimesemi_weights/vime_semi.ckpt


In [50]:
accuracy_score(y_test, pred)

0.9214